# Exercise 1 - Download, read and visualize H-SAF ASCAT SSM CDR and ISMN data

## Importing packages
Here we first import the python packages which we will need throughout this exercise. Don't worry about the warnings. 

In [ ]:
from pathlib import Path  # handling file paths
from typing import List  # type hinting

import matplotlib.pyplot as plt  # plotting data
import numpy as np
import pandas as pd  # data analysis library and dataframes (tabular data)
import pytesmo.temporal_matching as tmatch  # temporal matching of time series
import xarray as xr  # handling multi-dimensional arrays
import yaml  # YAML parser for configuration files
from ascat.read_native.ragged_array_ts import CellFileCollection  # ASCAT data
from ismn.components import Station  # type hinting
from ismn.interface import ISMN_Interface  # reading ISMN data
from ismn.meta import MetaData  # type hinting

## Set paths for data
Here we specify the paths to the directories, where our data is located. 

In [ ]:
# Get the root path from the yaml file
paths = yaml.safe_load(Path("../paths.yml").read_text())

# Set the root directory to all datasets
root: Path = Path(paths["jupyterhub"]).expanduser()

# Set the paths to the datasets
cell_source: Path = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
workspace: Path = root / "courses/120.110_Data-Retrieval-in-Earth-Observation"
ismn_path: Path = workspace / "ismn_data"

# Check if the paths exist
assert cell_source.exists()
assert ismn_path.exists()

## Read ISMN Data

In order to read the ISMN data, we are going to use the package `ismn`. This package is already installed in this environment.
For further information on how to use the package, please refer to the documentation: https://ismn.readthedocs.io/en/latest/examples/interface.html

The following cells show you how to read the data.

In [ ]:
# Loading the data from the provided Zip file
ismn_zip_file: Path = list(ismn_path.rglob("*.zip"))[0]
ismn_data: ISMN_Interface = ISMN_Interface(ismn_zip_file, parallel=False)

In [ ]:
# Aggregate data for the plot of the stations
station_ids: List[int] = []
lats: List[float] = []
lons: List[float] = []

for station in ismn_data.stations_that_measure("soil_moisture"):
    station_ids.append(station.name)
    lats.append(station.lat)
    lons.append(station.lon)

In [ ]:
# Plotting the locations of the available stations
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(
    lons, lats, marker="o", c="red", edgecolors="black", label="ISMN Stations"
)

for idx, station in enumerate(station_ids):
    ax.text(lons[idx], lats[idx], station, fontsize=9, ha="right", va="bottom")

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("ISMN Station Locations")
ax.legend()
ax.grid()
plt.show()

In [ ]:
# Inspect the soil moisture stations data
for station in ismn_data.stations_that_measure("soil_moisture"):
    print(f"\nStation '{station.name}' has the following sensors:")

    for sensor in station.sensors:
        print(f"\t- {sensor}")

    print("_" * 60)

In [ ]:
# Set some common parameters
network: str = "WEGENERNET"
sensor: str = "Hydraprobe-II_soil_moisture_0.200000_0.200000"

In [ ]:
# Inspect a single station's sensors
station_id: int = 54
station_obj: Station = ismn_data[network].stations.get(station_id)
print(station_obj)

In [ ]:
# Load the in-situ soil moisture data into an xarray dataset
insitu_ds: xr.Dataset = (
    ismn_data[network].stations.get(station_id)[sensor].to_xarray()
)
insitu_ds = insitu_ds.squeeze()
insitu_ds

Now that we have read the ISMN data into an xarray Dataset, we can visualize the data.

In [ ]:
insitu_ds.soil_moisture.plot()

In [ ]:
# Let's also check what our time series look like including the masked values
# Get the soil moisture and flag data
sm: xr.DataArray = insitu_ds.soil_moisture
sm_flag: xr.DataArray = insitu_ds.soil_moisture_flag

# Plot the soil moisture data
fig, ax = plt.subplots(figsize=(15, 5))
sm.plot(ax=ax, label="In-situ SM")
sm.where(sm_flag != "G").plot.scatter(
    ax=ax, c="crimson", ec="none", zorder=2, s=10, label="IN SITU SM FLAGGED"
)
ax.set_ylabel(r"Vol. Soil Moisture (m$^3$/m$^3$)")
ax.legend()
plt.show()

In [ ]:
# Extracting the metadata of the station
meta: MetaData = ismn_data["WEGENERNET"].stations.get(station_id).metadata
meta: pd.Series = meta.to_pd()
meta

In the next step we are going to store the coordinate data of the ISMN data in two separate arrays, one for the latitude and one for the longitude.

In [ ]:
lon: pd.Series = ismn_data.metadata.longitude.val
lat: pd.Series = ismn_data.metadata.latitude.val

## Read H121 - Metop ASCAT SSM CDR 12.5 km sampling
Then the data can be read using either longitude and latitude or per gridpoint.

For further information on how to use the package, please refer to the documentation:
https://ascat.readthedocs.io/en/latest/

In [ ]:
h121_reader = CellFileCollection.from_product_id(cell_source, "H121_V1.0")
h121_ds: xr.Dataset = h121_reader.read(
    coords=(float(lat.iloc[0]), float(lon.iloc[0]))
)
h121_ds = h121_ds.set_xindex("time").rename({"obs": "time"})
h121_ds

## Plot H121 Soil Moisture data
Now that we have loaded the data we can have a first look at the timeseries. Soil moisture is available as a percentage saturation. If you wish to calculate absolute soil moisture values, a possibility is to multiply the soil moisture data with the porosity value.

In [ ]:
# Plot the ASCAT H121 soil moisture data
fig, ax = plt.subplots(figsize=(15, 5))
h121_ds.surface_soil_moisture.plot(
    x="time", ax=ax, label="ASCAT H121 Soil Moisture"
)
ax.set_ylabel("Degree of Saturation (%)")
ax.legend()
plt.show()

In [ ]:
# Plot the soil moisture data for the year 2018
fig, ax = plt.subplots(figsize=(15, 5))
h121_ds.surface_soil_moisture.sel(time="2018").plot(
    x="time", ax=ax, label="ASCAT H121 Soil Moisture"
)
ax.set_ylabel("Degree of Saturation (%)")
ax.legend()
plt.show()

As discussed in the live session, soil moisture is affected by snow cover and frozen soils. We can see this in the subset of 2018, where a clear drop in soil moisture is visible in March, which is not likely due to changes in soil moisture. 
Hence we mask for frozen soils and the snow cover using the probability mask. Please note, this is not the same as masking using in situ data or model data, but only a probability of that these conditions occur.

## Calculate VOD for exercise 2
In exercise 2 you will need an estimate for TAU or the so-called Vegetation Optical Depth (VOD). TAU can be calculated from Metop ASCAT observables and this is done in the code below. VOD is written to the data frame of the ASCAT data, and when you temporally match the data with the ISMN data then you can store the dataarray including VOD.

In [ ]:
# define angles
cross_over_angle: int = 25
reference_angle: int = 40

# convert all backscatter to the dry reference cross over angle of 25
sig25: xr.DataArray = (
    h121_ds.backscatter40
    + h121_ds.slope40 * (cross_over_angle - reference_angle)
    + 0.5 * h121_ds.curvature40 * (cross_over_angle - reference_angle) ** 2.0
)
# get the lowest backscatter
dry_25: float = sig25.quantile(0.05).values.astype(float)

# convert to the reference angle of 40
dry_ref: xr.DataArray = (
    dry_25
    - h121_ds.slope40 * (cross_over_angle - reference_angle)
    - 0.5 * h121_ds.curvature40 * (cross_over_angle - reference_angle) ** 2
)

# get the wet reference
wet_ref: xr.DataArray = h121_ds.backscatter40.quantile(0.95)

# set bare soil sensitivity
lin_bs_sens: float = 0.21

# convert to linear domain
lin_wet_ref: xr.DataArray = 10 ** (wet_ref / 10.0)
lin_dry_ref: xr.DataArray = 10 ** (dry_ref / 10.0)

# get total backscatter sensitivity
lin_sens: xr.DataArray = lin_wet_ref - lin_dry_ref
inc_angle: np.float64 = -np.cos(np.radians(reference_angle)) / 2.0

# calculate vod as ratio between bare soil sensitivity
# and total backscatter sensitivity
# and store it in the h121 dataset
h121_ds["vod"] = inc_angle * np.log(lin_sens / lin_bs_sens)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

h121_ds.sel(time="2018").vod.plot(label="H121 VOD", ax=ax)

ax.set_ylabel(r"VOD (m$^2$/m$^2$)")
ax.legend()
plt.show()

## Temporal matching
The data of the in situ station and the satellite observations do not match. That means that the satellite data is available from 2017 to 2025, the in situ station only until 2024. The observation times are different with satellite observation irregular between 6 and 10 AM/PM and the in situ data at hourly intervals. So these two datasets need to be matched in time in order to have the same amount of observations. Here it is best practice to match to the most sparse dataset, in our case the ASCAT data.

We will use ``pytesmo`` - a Python Toolbox for the Evaluation of Soil Moisture Observations, which works with ``pandas`` dataframes. So the first step is to convert xarray datasets to pandas dataframes.

In [ ]:
# Xarray dataset to pandas dataframe
h121_df: pd.DataFrame = h121_ds.to_dataframe().reset_index()
insitu_df: pd.DataFrame = insitu_ds.to_dataframe().reset_index()

# Set index of dataframes to be time data
h121_df = h121_df.set_index("time")
insitu_df = insitu_df.set_index("date_time")

In [ ]:
# Find common date range
start_date: np.datetime64 = max(
    insitu_ds.date_time.min(), h121_ds.time.min()
).values
end_date: np.datetime64 = min(
    insitu_ds.date_time.max(), h121_ds.time.max()
).values

print(f"Common date range:\n\t{start_date}\n\t{end_date}")

In [ ]:
# Clip both dataframes to the date range
h121_df_clipped = h121_df.loc[start_date:end_date]
insitu_df_clipped = insitu_df.loc[start_date:end_date]

In [ ]:
# Temporal matching using pytesmo
# The window of 1 means that the maximum distance
# between the observations is 1 day

insitu_match = tmatch.temporal_collocation(
    h121_df_clipped, insitu_df_clipped, window=1
)

df = h121_df_clipped.join(insitu_match, how="inner")

Keep in mind that all variables are now in one dataset, where ``soil_moisture`` stands for in situ data, while ``surface_soil_moisture`` for ASCAT data. Therefore, we should rename variables for convenience.

In [ ]:
df = df.rename(
    columns={"soil_moisture": "SM_insitu", "surface_soil_moisture": "SM_ascat"}
)

# check if same number of observations
assert len(h121_df_clipped) == len(insitu_match)
print(f"Number of ASCAT observations: {len(h121_df_clipped)}")
print(f"Number of matched in-situ observations: {len(insitu_match)}")

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax2 = ax.twinx()

# Plot In-Situ soil moisture
lns1 = ax.plot(
    df.index,
    df["SM_insitu"],
    label="In-Situ SM",
    c="midnightblue",
    linestyle="-",
    linewidth=1.5,
    alpha=0.9,
)

ax.set_ylabel(r"In-Situ SM (m$^3$/m$^3$)", color="midnightblue")
ax.tick_params(axis="y", labelcolor="midnightblue")

# Plot ASCAT soil moisture
lns2 = ax2.plot(
    df.index,
    df["SM_ascat"],
    label="ASCAT SM",
    c="seagreen",
    linestyle="-",
    linewidth=1.5,
    alpha=0.5,
)

ax2.set_ylabel("ASCAT SM (Degree of Saturation %)", color="seagreen")
ax2.tick_params(axis="y", labelcolor="seagreen")

# Combine legends properly
legends = lns1 + lns2
labs = [legend.get_label() for legend in legends]
ax.legend(legends, labs, loc=2, fontsize=10, frameon=True)

plt.show()

We can use the provided flags from the in situ and satellite data sets to mask our data.

In [ ]:
# Mask out in-situ data anomalies
df_masked = df.loc[df["soil_moisture_flag"] == "G", :]

# Mask out frozen and snow-covered conditions
mask = (df_masked["frozen_soil_probability"] > 0) | (
    df_masked["snow_cover_probability"] > 0
)
df_masked = df.mask(mask)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax2 = ax.twinx()  # Secondary y-axis for ASCAT SM

# Plot In-Situ Soil Moisture (Blue)
ax.plot(
    df_masked.index,
    df_masked["SM_insitu"],
    label="In-Situ SM",
    color="midnightblue",
    linewidth=1.5,
    alpha=0.9,
)

# Plot ASCAT Soil Moisture (Green)
ax2.plot(
    df_masked.index,
    df_masked["SM_ascat"],
    label="ASCAT SM",
    color="seagreen",
    linewidth=1.5,
    alpha=0.5,
)

ax.set_ylabel("In-Situ SM (m³/m³)", color="midnightblue")
ax.tick_params(axis="y", labelcolor="midnightblue")

ax2.set_ylabel("ASCAT SM (Degree of Saturation %)", color="seagreen")
ax2.tick_params(axis="y", labelcolor="seagreen")

ax.legend(loc=2)
ax2.legend(loc=1)

plt.show()

Let's save the data to load it for other notebooks. The most convenient format to store the data in is called ``csv``.

In [ ]:
df_masked.to_csv("soilmoisture_vegetation_optical_depth.csv", index=True)

The data can also be saved as an xarray dataset, if that framework is preferred.

In [ ]:
df_masked = df_masked.dropna(how="all")

# Convert Pandas DataFrame to Xarray Dataset
ds_masked = xr.Dataset.from_dataframe(df_masked)

# Uncomment line below if you wish to save an xarray dataset
# ds_masked.to_zarr("soilmoisture_vegetation_optical_depth.zarr", mode="w")

ds_masked

## Exercise
To complete the exercise, please locate an ISMN station of your interest, download the ISMN data, read in the data using the longitude and latitude. Do a temporal matching of the data and plot the time series as shown below and upload this to the Padlet with a description and first interpretation. If necessary, save your matched dataframe (see below) so you have it ready for the other exercises.